In [53]:
import dgl
import numpy as np
import torch
import torch.nn as nn
from scipy import sparse
import networkx as nx
from scipy.sparse import diags
import torch.nn.functional as F
from pathlib import Path

import json
import time
from pathlib import Path
from networkx.readwrite import json_graph, read_gpickle
from networkx.linalg.laplacianmatrix import laplacian_matrix
from scipy.io import mmwrite, mmread
from scipy.sparse import csr_matrix
import sklearn
from scipy.io import mmread, mmwrite # spare save/load
from collections import Counter, defaultdict
from tqdm import trange, tqdm
import pdb

# Utils Functions

In [3]:
# a = np.array([1, 0, 3])
from scipy import sparse
def onehot(a): 
    b = np.zeros((a.size, a.max()+1))
    b[np.arange(a.size),a] = 1
    return b
# onehot(a)

def load_matrix(graph):
    if 'networkx' in str(type(graph)):
        adj_matrix = nx.adj_matrix(graph)
    else:
        adj_matrix=graph
    degree_vec = adj_matrix.sum(axis=1).astype(np.float)
    with np.errstate(divide='ignore'):
        d_inv_sqrt = np.squeeze(np.asarray(np.power(degree_vec, -1)))
    d_inv_sqrt[np.isinf(d_inv_sqrt) | np.isnan(d_inv_sqrt)] = 0
    degree_matrix = diags(d_inv_sqrt, 0)
    return adj_matrix, degree_matrix

def lpa(adj_matrix,degree_matrix, labels,train_mask,test_mask, iteration=10):
    influence=labels.copy()
    influence[np.arange(train_mask.size,labels.shape[0])]=0  # remove invisible_nodes
    for _ in range(iteration):
        influence = degree_matrix@adj_matrix@influence
        influence[train_mask]=labels[train_mask]
    pred=influence.argmax(1)
    labels=labels.argmax(1)
    border_nodes = (pred!=labels).nonzero()[0]
    acc = (pred[test_mask]==labels[test_mask]).sum()/labels[test_mask].size
    return influence, acc, border_nodes, influence

# influence, acc =lpa(adj_matrix,degree_matrix,onehot_labels,train_mask,iteration=20)

## Load Data

In [4]:
from dgl.data import citation_graph as citegrh
cora = citegrh.load_cora()
citeseer = citegrh.load_citeseer()
pubmed = citegrh.load_pubmed()
reddit = dgl.data.RedditDataset()

/data/data0/yushi/dgl_env/lib/python3.7/site-packages/dgl/data/citation_graph.py:140: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


Finished data loading and preprocessing.
  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Finished data loading and preprocessing.
  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Finished data loading.
  NumNodes: 232965
  NumEdges: 114615892
  NumFeats: 602
  NumClasses: 41
  NumTrainingSamples: 153431
  NumValidationSamples: 23831
  NumTestSamples: 55703


In [98]:
data=reddit
dataset='reddit'
graph=data.graph
labels=data.labels
onehot_labels=onehot(data.labels)
train_mask=data.train_mask.astype(np.int).nonzero()[0]
test_mask=data.test_mask.astype(np.int).nonzero()[0]
train_labels = labels[train_mask]
onehot_labels = F.one_hot(torch.LongTensor(labels)).numpy()
levels = 4
reduce_results = f"graphzoom/reduction_results/{dataset}/no_fusion/"

In [99]:
# for reddit

dataset_prefix='/data/data0/yushi/'
dataset_dir=f'{dataset_prefix}/dataset/{dataset}'
npz_path = Path(f'{dataset_dir}/{dataset}.npz')
graph = sparse.load_npz(str(npz_path))


## Run LPA

In [111]:
adj_matrix, degree_matrix=load_matrix(graph)
# %timeit 
influence, acc, border_nodes, influence =lpa(adj_matrix,degree_matrix,onehot_labels,train_mask,test_mask,iteration=20)

In [113]:
len(border_nodes)

5942

## Coarsen Analysis

In [91]:

from graphzoom.utils import construct_proj_laplacian, mtx2matrix
def check_agg(nodes, projection):
    coarsen_nodes=[]
    for node in nodes:
        seed=projection[:,node].nonzero()[0][0]
        agg_nodes = projection[seed].nonzero()[1]
        if agg_nodes.size>1:
            coarsen_nodes.append(node)
    return coarsen_nodes

def onehot_proj(projection, train_mask):
#     seed=projection[:,node].nonzero()[0][0]
    col, row= [], []
    for seed in trange(projection.shape[0]):
        agg_nodes = projection[seed].nonzero()[1]
#         print(agg_nodes)
        train_nodes = [node for node in agg_nodes if node in train_mask]
        unknown_nodes = [node for node in agg_nodes if node not in train_nodes]
        if len(train_nodes): # 
            agg_group=defaultdict(list)
            for node in train_nodes:
                agg_group[labels[node]].append(node)
            most_common=sorted(agg_group.items(), key=lambda x:len(x[1]), reverse=True)[0][0]
            agg_group[most_common]+=unknown_nodes
#             if len(agg_group)>1:
#                 pdb.set_trace()
            for hypter_node in agg_group:
                row+=[row[-1]+1 if len(row) else 0]*len(agg_group[hypter_node])
                col+=agg_group[hypter_node]
        else: # feed unknown
            row += [row[-1]+1 if len(row) else 0] * len(unknown_nodes)
            col += unknown_nodes
            
#     pdb.set_trace()
    assert len(col)==projection.shape[1]
    data = np.ones(projection.shape[1])
    proj = sparse.coo_matrix((data, (row, col)))
    return proj

In [103]:
input_path = "graphzoom/dataset/{}/{}.mtx".format(dataset, dataset)
print('loading mtx')
laplacian = mmread(input_path)
print('loading finished')
projections, coarse_adj = construct_proj_laplacian(laplacian, levels, reduce_results)
projections

loading mtx
loading finished


[<82494x232965 sparse matrix of type '<class 'numpy.int64'>'
 	with 232965 stored elements in Compressed Sparse Row format>,
 <30592x82494 sparse matrix of type '<class 'numpy.int64'>'
 	with 82494 stored elements in Compressed Sparse Row format>,
 <11865x30592 sparse matrix of type '<class 'numpy.int64'>'
 	with 30592 stored elements in Compressed Sparse Row format>,
 <4888x11865 sparse matrix of type '<class 'numpy.int64'>'
 	with 11865 stored elements in Compressed Sparse Row format>]

1. one_hot -> merge
2. multi_hot -> merge in separate
3. unknown -> most?

In [92]:
# train_mask
proj = [onehot_proj(projections[0], train_mask)]
# d={1:[1,2,3],2:[1,2,3,3,3,3,]}
# sorted(d.items(), key=lambda x:len(x[1]), reverse=True)1

100%|██████████| 82494/82494 [00:54<00:00, 1509.34it/s]


In [110]:
# border_nodes

array([ 144,  147,  151,  153,  154,  156,  159,  172,  174,  176,  178,
        181,  194,  207,  214,  218,  223,  225,  226,  227,  228,  236,
        240,  242,  244,  245,  246,  247,  256,  259,  261,  262,  263,
        266,  269,  272,  280,  282,  286,  291,  299,  303,  304,  310,
        311,  314,  317,  318,  319,  320,  321,  322,  338,  340,  341,
        346,  364,  365,  366,  371,  377,  378,  379,  382,  383,  396,
        400,  401,  414,  417,  418,  424,  429,  431,  434,  441,  442,
        444,  445,  449,  450,  469,  474,  479,  485,  487,  493,  496,
        500,  501,  503,  504,  506,  508,  509,  510,  516,  518,  522,
        523,  524,  525,  528,  529,  547,  551,  563,  571,  584,  587,
        589,  590,  592,  596,  604,  617,  622,  624,  626,  627,  631,
        632,  634,  639,  646,  651,  658,  659,  662,  663,  668,  669,
        673,  677,  678,  682,  683,  689,  690,  694,  695,  700,  702,
        709,  711,  720,  723,  740,  750,  754,  7

In [114]:
coarse_nodes = check_agg(border_nodes,projections[0])
print(len(coarse_nodes), len(border_nodes))
from random import shuffle
shuffle(coarse_nodes)

5201 5942


In [496]:
def add_border_nodes_to_proj_matrix(projection, nodes):
    # convert to csc
    projs=[]
    if type(projection) is not list:
        projection=[projection]
    # change first projection
    proj=projection[0].tocsc()
    print(proj.shape)
    next_level_size=proj.shape[0]
    for node in nodes:
        proj.indices[node]=next_level_size
        next_level_size+=1
    projs.append(sparse.csc_matrix((proj.data, proj.indices,proj.indptr), dtype=np.longlong))
    # change via coo_matrix
    for i in range(1, len(projection)):
        proj = projection[i].tocoo()
        data = np.ones(proj.shape[1]+len(nodes)).astype(np.longlong)
        col = np.hstack((proj.col, np.arange(proj.shape[1],proj.shape[1]+len(nodes))))
        row = np.hstack((proj.row, np.arange(proj.shape[0],proj.shape[0]+len(nodes))))
        projs.append(sparse.coo_matrix((data, (row, col))))
    
    return projs

In [594]:
coarse_nodes_left=1000
border_proj=add_border_nodes_to_proj_matrix(projections[:], coarse_nodes[:coarse_nodes_left])
proj = border_proj
border_proj

(82494, 232965)


[<83494x232965 sparse matrix of type '<class 'numpy.longlong'>'
 	with 232965 stored elements in Compressed Sparse Column format>,
 <31592x83494 sparse matrix of type '<class 'numpy.longlong'>'
 	with 83494 stored elements in COOrdinate format>,
 <12865x31592 sparse matrix of type '<class 'numpy.longlong'>'
 	with 31592 stored elements in COOrdinate format>,
 <5888x12865 sparse matrix of type '<class 'numpy.longlong'>'
 	with 12865 stored elements in COOrdinate format>]

In [96]:
def overwrite(path):
    with open(path, 'r+') as f:
        content = f.readlines()[2:]
        f.seek(0)
        f.writelines(content)
        f.truncate()
from graphzoom.utils import mtx2matrix
prefix=Path(f"graphzoom/reduction_results/{dataset}/one_hot/")
if not prefix.exists():
    prefix.mkdir(parents=True)
for i in range(len(proj)):
    mmwrite(str(prefix.joinpath(f'Projection_{i+1}.mtx')), proj[i])
    overwrite(str(prefix.joinpath(f'Projection_{i+1}.mtx')))
# with open(str(prefix.joinpath(f'NumLevels.txt')), 'w') as f:
#     f.write(str(len(border_proj)))

reduce_results = f"graphzoom/reduction_results/{dataset}/border/"
border_projs, border_coarse_adj = construct_proj_laplacian(laplacian, 4, reduce_results)
mmwrite(str(prefix.joinpath(f'Gs.mtx')), border_coarse_adj[3],symmetry='symmetric')
overwrite(str(prefix.joinpath(f'Gs.mtx')))

In [598]:
border_coarse_adj

[<232965x232965 sparse matrix of type '<class 'numpy.int64'>'
 	with 114848857 stored elements in COOrdinate format>,
 <83494x83494 sparse matrix of type '<class 'numpy.longlong'>'
 	with 58028143 stored elements in Compressed Sparse Row format>,
 <31592x31592 sparse matrix of type '<class 'numpy.longlong'>'
 	with 28021941 stored elements in Compressed Sparse Row format>,
 <12865x12865 sparse matrix of type '<class 'numpy.longlong'>'
 	with 11493302 stored elements in Compressed Sparse Row format>]

In [507]:
(border_projs[0]!=projections[0])

True

In [314]:
g = mtx2matrix(str(prefix.joinpath(f'Gs.mtx')))

In [373]:
tg=nx.Graph()
# tg.add_nodes_from([1,2,3])
tg.add_edge(1,2)
tg.add_edge(2,1)